# Descarga datos esios

## Documentacion:

La nueva web pública de e·sios pone a disposición de todos los usuarios una API para la descarga de información, cuya documentación se encuentra disponible en https://api.esios.ree.es.

Para poder utilizar esta API deberán solicitar un token personal enviando un correo a **consultasios@ree.es**.En mi caso:

token: 9d6bcd627698602fbcd18721cca88b90d0f6e6025963f86be84fa18c87801a10

En nuestro caso, la solicitud de datos será para un periodo de tiempo solicitando los datos por código indicador:

https://api.esios.ree.es/indicator/getting_a_specific_indicator_filtering_values_by_a_date_range


## 1. Importamos librerías necesarias

In [1]:
import pandas as pd
import numpy as np
import requests
import json

In [2]:
pd.options.display.max_columns= None

In [3]:
requests.get?

Signature: requests.get(url, params=None, **kwargs)
Docstring:
Sends a GET request.

:param url: URL for the new :class:`Request` object.
:param params: (optional) Dictionary or bytes to be sent in the query string for the :class:`Request`.
:param \*\*kwargs: Optional arguments that ``request`` takes.
:return: :class:`Response <Response>` object
:rtype: requests.Response
File:      /anaconda3/lib/python3.6/site-packages/requests/api.py
Type:      function


## 2. Definición features

Creamos diccionario

In [4]:
d={10010: 'P48_EOLICA',
   10027: 'P48_DEMANDA',
   10008: 'P48_CARBON',
   475: 'POT_DISP_HULLA_ANT',  #Los tendremos que sumar para obtener MW disponibles Carbón
   476: 'POT_DISP_HULA_SB',
   600: [['PRECIO_MD_ESP',3],['PRECIO_MD_FR',2]],
   612: [['PRECIO_I1',3]]
  }

In [5]:
d.items()

dict_items([(10010, 'P48_EOLICA'), (10027, 'P48_DEMANDA'), (10008, 'P48_CARBON'), (475, 'POT_DISP_HULLA_ANT'), (476, 'POT_DISP_HULA_SB'), (600, [['PRECIO_MD_ESP', 3], ['PRECIO_MD_FR', 2]]), (612, [['PRECIO_I1', 3]])])

En el caso del id_precios habrá que adentrarse un nivel adicional y consultar GEOIDS:

* ESPAÑA = 3
* FRANCIA = 2

## 3. Funciones solicitud datos

Función para obtener los datos del indicador *ind* solicitado entre dos fechas dadas.

En nuestro dataset se nos presentan 2 peculiaridades a introducir en la función:

1) Existe un único indicador para los precios de cada mercado, diferenciándose el pais por el parámetro GEOIDS --> **solicita_datos_precios**

2) Los datos de potencia disponible hay que agregarlos porque viene desglosada en comunidades --> **solicita datos disponibles**

In [6]:
def solicita_datos_general(fecha_ini,fecha_fin,ind):

    headers_={'Accept': 'application/json; application/vnd.esios-api-v1+json',
              'Content-Type': 'application/json',
              'Host': 'api.esios.ree.es',
              'Authorization': 'Token token="9d6bcd627698602fbcd18721cca88b90d0f6e6025963f86be84fa18c87801a10"',
              'Cookie':''
             }

    query_parametros_={'start_date': fecha_ini,
                'end_date':fecha_fin}

    url_='https://api.esios.ree.es/indicators/' + str(ind)
    
    #Hacemos peticion:
    r = requests.get(url_,headers=headers_,params=query_parametros_)
    
    #Comprobamos que trae datos
    assert r.status_code==200
    
    #r.content --> durante el diseño para explorar datos
    
    data=json.loads(r.content)
    
    #type(data)
    #data['indicator']['values'] --> durante el diseño para explorar datos
    
    df_data=pd.DataFrame(data['indicator']['values'])
    
    #df_data.dtypes --> Tenemos que convertir las fechas a formato fecha
    #y crear nuestro df
    
    df=pd.DataFrame()        
    df['datetime']=pd.to_datetime([f[:10] + ' ' + f[11:13] + ':00:00' for f in df_data['datetime']]) 
    df['value']=df_data['value']
    if (ind==475) | (ind==476):
        df=df.groupby('datetime')['value'].sum()
        df=pd.DataFrame(df)
    else:
        df=df.set_index('datetime')

    return df

In [7]:
def solicita_datos_precios(fecha_ini,fecha_fin,ind,lista):
   
    headers_={'Accept': 'application/json; application/vnd.esios-api-v1+json',
              'Content-Type': 'application/json',
              'Host': 'api.esios.ree.es',
              'Authorization': 'Token token="9d6bcd627698602fbcd18721cca88b90d0f6e6025963f86be84fa18c87801a10"',
              'Cookie':''
             }
 
    query_parametros_={'start_date': fecha_ini,
                       'end_date': fecha_fin}
 
    url_='https://api.esios.ree.es/indicators/'+ str(ind)
 
    #Hacemos peticion:
    r = requests.get(url_,headers=headers_,params=query_parametros_)
 
    #Comprobamos que trae datos
    assert r.status_code==200
 
    data=json.loads(r.content)
 
    data['indicator']['values']
 
    df_data=pd.DataFrame(data['indicator']['values'])
 
    df=pd.DataFrame()
    for i in enumerate(lista):
        df_subset=pd.DataFrame()
        df_data_subset=df_data[df_data['geo_id']==i[1][1]]
        df_data_subset.reset_index(drop=True,inplace=True)
        df_subset['datetime']=pd.to_datetime([f[:10] + ' ' + f[11:13] + ':00:00' for f in df_data_subset['datetime']])
        df_subset[str(i[1][0])]=df_data_subset['value']
        df_subset.set_index('datetime',inplace=True)
        df=pd.concat([df,df_subset],axis=1)
    return df      
 

## 4. Agregación dataset

In [8]:
datos_esios=pd.DataFrame()  #será nuestro set de datos

fecha_ini_='2015-10-01 00:00:00'
fecha_fin_='2018-09-30 23:59:59'

Definimos parámetros de entrada:

La idea inicial es que mi dataset empiece en 01/10/2015 hasta 30/09/2018.

He escogido 3 años móviles, teniendo en cuenta que el año hidráulico comienza el octubre (y la hidraulicidad es importante en el Mercado eléctrico) e intentar evitar el efecto de cambios regulatorios en la mayor medida posible

In [9]:
for k,v in d.items():
    if (k==600) | (k==612):
        historico=solicita_datos_precios(fecha_ini_,fecha_fin_,k,v)
    else:    
        historico=solicita_datos_general(fecha_ini_,fecha_fin_,k)
        historico.columns=[str(v)]
    datos_esios=pd.concat([datos_esios,historico],axis=1)
    

In [10]:
datos_esios.head()

,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1
datetime,,,,,,,,
2015-10-01 00:00:00,2196.1,24517.1,8203.3,5355.4,4705.3,56.65,38.56,56.31
2015-10-01 01:00:00,2064.4,23168.6,8286.8,5355.4,4705.3,53.50,31.27,54.25
2015-10-01 02:00:00,1866.5,22437.2,8301.3,5355.4,4705.3,49.69,30.66,49.69
2015-10-01 03:00:00,1660.9,22281.3,8281.3,5355.4,4705.3,49.10,29.14,49.10
2015-10-01 04:00:00,1373.9,21909.7,8247.7,5355.4,4705.3,51.25,29.40,51.25


## 5. Check datos

In [11]:
datos_esios.describe()

,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1
count,26304.000000,26304.000000,26304.000000,26304.000000,26304.000000,26304.000000,26304.000000,26304.00000
mean,5479.286014,28672.990169,4409.936025,4420.339625,4188.581794,48.739281,42.105033,48.46574
std,3185.927399,4616.496856,2216.248299,474.038209,469.363759,14.954689,20.799903,15.09524
min,240.800000,17728.400000,546.000000,2461.500000,2315.100000,2.060000,-31.820000,0.00000
25%,3018.400000,24734.150000,2316.975000,4125.800000,3797.600000,40.137500,29.740000,40.05000
50%,4822.950000,28920.600000,4634.000000,4483.200000,4148.100000,49.550000,39.165000,49.00000
75%,7289.250000,32267.650000,6282.750000,4799.700000,4705.300000,59.450000,51.562500,58.90000
max,16952.600000,41297.900000,8768.300000,9481.400000,8270.600000,101.990000,874.010000,102.49000


In [12]:
datos_esios.shape  #para testear que no tengo NA

(26304, 8)

## 6. Salvar datos

In [13]:
datos_esios.to_csv('../data/datos_esios.csv',sep=';')